# 使用高速过滤器标注全部数据

In [1]:
#!AHOCORASICK_BYTES=1 pip install git+https://github.com/WojciechMula/pyahocorasick.git

In [2]:
import os
import json
import random
from matplotlib import pyplot as plt
import numpy as np
from tokenizer import tokenizer,token2str,vocab_size
import torch
import torch.nn as nn
from make_model import make_model
from train_and_use import Batch,CrossEntropyLoss,SimpleAdamOptimizer,OptimizerWrapper,train_server_start
from train_and_use import greedy_decode #使用text_continue时重复惩罚会严重减速，换用greedy_decode
from train_and_use import TOGGLE,STOP
from train_and_use import record
from tqdm import tqdm
import concurrent.futures
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# %matplotlib widget

In [3]:
model = make_model(
    #token是从1开始的，0填充，剩下的用来覆盖全部字节
    vocab_size = vocab_size+1+255,
    embedding_dim = 16,
    key_dim = 8,
    head_number = 2,
    position_information_type = "mask",
    # position_information_type = "sinusoidal",
    # position_information_type = "rotary",
    # position_information_type = "learned",
    enable_affine = True,
    enable_talking_head = False,
    use_diff = False,
    self_attention_block_size = 0,
    feed_forward_dim = 16,
    enable_layer_norm = True,
    deep = 1,
    dropout_rate = 0.1
).to(device)

In [4]:
model.load_state_dict(torch.load('fastautotagger.weight',weights_only=True))
model = model.eval()

In [5]:
file_high = open('high_ff.txt','a')
file_mid = open('mid_ff.txt','a')
file_low  = open('low_ff.txt','a')

In [6]:
fnames = os.listdir('WuDaoCorpus2.0_base_200G')
fnames.sort()
cnt = 0
prompt = tokenizer(' 这段文本的质量按照“高、中、低”三档评价为：',5.0)
tokens_batch = []
data_ff = []
for fname in fnames:
    cnt += 1
    print('\r[',cnt,'/',len(fnames),']',end=' '*10)
    with open('WuDaoCorpus2.0_base_200G/'+fname,'r',encoding='utf-8') as f:
        lines = [item['content'] for item in json.load(f)]
        with torch.amp.autocast("cuda"):
            with concurrent.futures.ProcessPoolExecutor(max_workers=14) as executor:
                task_dict = {executor.submit(tokenizer,line,5):line for line in lines}
                with tqdm(total=len(task_dict)) as pbar:
                    for future in concurrent.futures.as_completed(task_dict):
                        data_ff += [task_dict[future]]
                        tokens_batch += [(future.result()+[-255]*1000)[:1000] + prompt]
                        if len(tokens_batch) == 256:
                            tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
                            inputs = torch.from_numpy(tokens_batch).to(device).data
                            with torch.no_grad():
                                o = greedy_decode(model,inputs,out_length=1)
                            for res,line in zip(o,data_ff):
                                res = token2str(res.cpu().numpy()-255)[-1]
                                if res == '高':
                                    print(line,file=file_high)
                                if res == '中':
                                    print(line,file=file_mid)
                                if res == '低':
                                    print(line,file=file_low)
                            tokens_batch = []
                            data_ff = []
                        pbar.update(1)
            if len(tokens_batch) != 0:
                tokens_batch = np.array(tokens_batch,dtype=np.int64)+255
                inputs = torch.from_numpy(tokens_batch).to(device).data
                with torch.no_grad():
                    o = greedy_decode(model,inputs,out_length=1)
                for res,line in zip(o,data_ff):
                    res = token2str(res.cpu().numpy()-255)[-1]
                    if res == '高':
                        print(line,file=file_high)
                    if res == '中':
                        print(line,file=file_mid)
                    if res == '低':
                        print(line,file=file_low)
                tokens_batch = []
                data_ff = []

[ 1 / 366 ]          

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 39888/39888 [00:54<00:00, 737.91it/s]


[ 2 / 366 ]          

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 49679/49679 [00:56<00:00, 871.87it/s]


[ 3 / 366 ]          

...

[ 66 / 366 ]          

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 781280/781280 [15:39<00:00, 831.33it/s]


[ 67 / 366 ]          

100%|██████████████████████████████████████████████████████████████████████████████████████████████████▉| 809749/809753 [19:36<00:00, 688.34it/s]


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [7]:
file_high.close()
file_mid.close()
file_low.close()